# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [1]:
# %pip install ..

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
df = pd.read_csv("demonstration.csv")
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1933-03-04,12:41:26,2022-08-02 16:42:16
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1923-05-05,14:08:00,2022-08-10 13:32:45
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1925-12-06,10:51:26,2022-08-14 19:51:11
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1923-09-07,17:05:01,2022-07-30 08:22:05
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1919-12-07,14:18:06,2022-07-24 07:45:26


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv("demonstration.csv", dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1933-03-04,12:41:26,2022-08-02 16:42:16
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1923-05-05,14:08:00,2022-08-10 13:32:45
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1925-12-06,10:51:26,2022-08-14 19:51:11
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1923-09-07,17:05:01,2022-07-30 08:22:05
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1919-12-07,14:18:06,2022-07-24 07:45:26


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1933, 3, 4)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:126: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)
meta_dataset["Married since"].to_dict()
type(meta_dataset["Married since"].distribution.start)

pandas._libs.tslibs.timestamps.Timestamp

As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()
type(meta_dataset["Married since"].distribution.start)

pandas._libs.tslibs.timestamps.Timestamp

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"uq~X7r< /GT|""KkD{8 hB5B j 1BB5}wyD (I !i|kji~...",male,74,0,24.875056,<NA>,C,1906-03-22,16:07:37,2022-07-27 09:25:50
1,2,6D?/B C$nt:uF}9z&57jcVqnb}#Xii@)1OM jG#D08^EHm...,female,39,0,8.467339,<NA>,S,1922-12-18,17:03:11,2022-07-23 18:55:36
2,3,"zx?m~Auuw6kP""3LeG(d+.px",female,41,0,25.801449,<NA>,S,1933-02-04,17:50:07,2022-08-02 13:17:17
3,4,"6-h.""6+lsJ42EV""$ #;_!I a{ &j0",female,56,0,9.714752,<NA>,Q,1938-02-17,18:10:38,2022-07-26 09:31:42
4,5,"b:<c?E7 MHuAah]MV }?)fU@ o,L{@<mZDc([yP;B+GM|G...",male,0,0,15.883305,"DMDP^U,g{kI",Q,1914-03-28,12:12:31,2022-07-27 03:07:54


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Wayne Brown,male,17,0,10.619478,<NA>,S,1905-04-22,15:35:50,2022-08-08 15:42:56
1,2,Douglas Robinson,male,48,2,7.151268,<NA>,S,1927-05-10,15:49:38,2022-08-01 04:20:30
2,3,Albert Moore,male,76,2,11.014565,<NA>,C,1915-12-10,11:54:01,2022-08-13 11:09:46
3,4,Steven Reed,female,67,0,0.133778,<NA>,S,1933-02-13,18:01:43,2022-07-28 22:21:15
4,5,Robyn Walker,female,<NA>,0,3.410311,<NA>,S,1911-03-30,10:45:31,2022-08-05 21:22:49


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Thomas Rhodes,female,24,0,8.236637,<NA>,Q,1927-12-15,15:07:34,2022-08-07 04:33:43
1,2,Marcus Bailey,male,<NA>,0,0.814323,<NA>,Q,1938-05-14,13:21:52,2022-07-18 21:22:17
2,3,John Newton,male,<NA>,1,1.404709,Tn[3z.'tg,S,1910-01-02,16:46:42,2022-07-15 22:31:10
3,4,Brian Bryan,female,28,0,0.721040,EbbBV+,C,1904-08-26,11:10:15,2022-07-27 09:51:33
4,5,Jackson Nguyen,male,38,0,0.806611,<NA>,S,1922-04-16,16:21:24,2022-08-06 08:00:21


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution([(r"[ABCDEF]", 1), (r"\d{2,3}", 0.9)])  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Donna Guzman,male,25,0,0.369164,<NA>,S,1909-09-15,14:12:15,2022-07-21 12:25:04
1,2,Christy Eaton,female,29,0,0.569691,<NA>,C,1924-08-14,12:30:36,2022-07-23 16:58:09
2,3,William Little,female,<NA>,0,2.998094,<NA>,S,1909-10-11,18:17:04,2022-08-13 09:15:57
3,4,Carl Wright,male,<NA>,0,0.261898,<NA>,C,1927-03-13,11:13:16,2022-07-17 06:58:06
4,5,Austin Browning,male,<NA>,0,0.692547,<NA>,S,1923-10-14,11:22:44,2022-08-06 16:40:33
